In [51]:
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon

In [52]:
raw_co2_data=pd.read_csv('data/owid-co2-data.csv')
raw_co2_data

,iso_code,country,year,co2,co2_growth_prct,co2_growth_abs,consumption_co2,trade_co2,trade_co2_share,co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,primary_energy_consumption,energy_per_capita,energy_per_gdp,population,gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,NaN,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7663783.0,NaN
1,AFG,Afghanistan,1950,0.084,475.000,0.070,NaN,NaN,NaN,0.011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7752000.0,1.949480e+10
2,AFG,Afghanistan,1951,0.092,8.696,0.007,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7840000.0,2.006385e+10
3,AFG,Afghanistan,1952,0.092,NaN,NaN,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7936000.0,2.074235e+10
4,AFG,Afghanistan,1953,0.106,16.000,0.015,NaN,NaN,NaN,0.013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8040000.0,2.201546e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,ZWE,Zimbabwe,2015,12.170,1.653,0.198,13.308,1.138,9.350,0.881,...,4.885,11.87,0.859,6.68,0.484,NaN,NaN,NaN,13815000.0,2.503057e+10
23704,ZWE,Zimbabwe,2016,10.815,-11.139,-1.356,12.171,1.356,12.542,0.771,...,4.703,11.92,0.850,6.55,0.467,NaN,NaN,NaN,14030000.0,2.515176e+10
23705,ZWE,Zimbabwe,2017,10.247,-5.251,-0.568,11.774,1.527,14.902,0.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14237000.0,NaN
23706,ZWE,Zimbabwe,2018,11.341,10.674,1.094,12.815,1.475,13.006,0.785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14439000.0,NaN


Then we need to sort out the countries and select only a single value for each. We'll choose the latest.

In [53]:
nations=pd.unique(raw_co2_data['iso_code'])
for nat_i in nations:
    subset=raw_co2_data[raw_co2_data['iso_code']==nat_i]
    last_year=subset[subset['year']==subset['year'].max()]
    if nat_i == 'AFG':
        co2_data = last_year
    else:
        co2_data = co2_data.append(last_year,ignore_index=False)    


co2_data['iso_code']

70            AFG
293           ALB
397           DZA
427           AND
497           AGO
           ...   
23180         WLF
23450    OWID_WRL
23520         YEM
23590         ZMB
23707         ZWE
Name: iso_code, Length: 214, dtype: object

In [ ]:
subset=raw_co2_data[raw_co2_data['iso_code']=='OWID_WRL']
subset

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

In [ ]:
print(world.shape)
world.plot()

In [ ]:
arg_polygons=world.geometry
arg_polygons

In [ ]:
exploded = world.explode()
print(exploded.shape)
exploded.plot()

Unfortunately, then running the PyCartogram code on this exploded GeoDataFrame results is the following warning:
> ValueError: A LinearRing must have at least 3 coordinate tuples
The implication is that there is something duff that needs to be fixed in process of exploding.

In [ ]:
item0=exploded.iat[0,5]
print(item0)


In [ ]:
#Check Geometry
def npts(geom):
    LinRing=geom.exterior
    coords=LinRing.coords
    num_points=len(coords)
    return num_points

exploded['npts'] = exploded['geometry'].apply(lambda x: npts(x))
exploded['npts'].min()

There is a function written by JoeryJoery that will update the polygons of a GeoSeries in contiguous area form. It's available at <https://github.com/joeryjoery/PyCartogram>. I'm copying it below

In [ ]:
def cartogram(arg_polygons, arg_values, itermax=5, max_size_error=1.0001, epsilon=0.01, verbose=False):
    """
    Generate an area equalizing contiguous cartogram based on the algorithm by (J. Oougenik et al., 1985).
    
    Note: The current function does not include interior boundaries when distorting the polygons!
          This is due to shapely's current way of extracting boundary coordinates which make it 
          cumbersome to separate interior points from exterior points.
    
    :param arg_polygons: geopandas.geoseries.GeoSeries Series of shapely.geometry.Polygon or Multipolygon objects.
    :param arg_values: (geo)pandas.Series Series of floating point values.
    :param itermax: int (Optional, default=5) Maximum amount of iterations to perform adjusting coordinates.
    :param max_size_error: float (Optional, default=1.0001) A maximum accuracy until terminating the procedure.
    :param epsilon: float (Optional, default=0.01) Scalar to prevent zero division errors.
    :param verbose: bool (Optional, default=False) Whether to print out intermediary progress. 
    
    :returns: geopandas.geoseries.GeoSeries Copy of :arg_polygons: with the adjusted coordinates.
    
    :references: Dougenik, J.A., Chrisman, N.R. and Niemeyer, D.R. (1985), 
                 AN ALGORITHM TO CONSTRUCT CONTINUOUS AREA CARTOGRAMS*. 
                 The Professional Geographer, 37: 75-81. doi:10.1111/j.0033-0124.1985.00075.x 
    
    :see: Implementation of the same algorithm in R (available on CRAN): https://github.com/sjewo/cartogram
    """    
    
    arg_polygons.to_crs('+proj=cea')
    geometry = arg_polygons.copy().values
    values = arg_values.copy().values
    
    total_value = values.sum()
    mean_size_error = 100
    
    for iteration in range(itermax):
        if mean_size_error < max_size_error:
            break
        
        # This statement unpacks the centroid Point object to np.array and
        # creates a n x 2 matrix of centroid [x, y] coordinates.
        centroids = np.array(list(map(np.array, geometry.to_crs('+proj=cea').centroid.to_crs(geometry.crs))))
        area = geometry.to_crs('+proj=cea').area
        total_area = area.sum()
        
        desired = total_area * values / total_value
        desired[desired == 0] = epsilon  # Prevent zero division.
        radius = np.sqrt(area / np.pi)
        mass = np.sqrt(desired / np.pi) - np.sqrt(area / np.pi)
        
        size_error = np.max([desired, area], axis=0) - np.min([desired, area], axis=0)
        mean_size_error = np.mean(size_error)
        force_reduction_factor = 1 / (1 + mean_size_error)
        
        if verbose:
            print("Mean size error at iteration {}: {}".format(iteration+1, mean_size_error))
        for row, item in enumerate(geometry):
            print(row)
            # TODO: Possibly include shapely.geometry.Polygon interior coordinates.
            
            # Some coordinates may appear twice, however, they mustn't be removed.
            # These coordinates are also adjusted, but only computed once:
            coordinates = np.matrix(item.exterior.coords)    # [[x1, y2], [x2, y2], ...]
            idx = np.unique(coordinates, axis=0)                # Get unique rows
            
            for k in range(len(idx)):
                # Get positions from coordinates for each unique idx.
                coord_idx = np.where((coordinates[:, 0] == idx[k,0]) & (coordinates[:,1] == idx[k, 1]))[0]
                # Only extract one using coord_idx[0] as coord_idx maps duplicate coordinates.
                new_coordinates = coordinates[coord_idx[0],:]  

                # Compute coordinate's euclidean distances to all centroids.
                distances = np.sqrt(np.square(centroids - new_coordinates).sum(axis=1))
                distances = np.array(distances).ravel()  # Converts matrix into flat array.
                
                # Compute force vectors
                Fijs = mass * radius / distances
                Fbij = mass * np.square(distances / radius) * (4 - 3 * distances / radius)
                Fijs[distances <= radius] = Fbij[distances <= radius]
                Fijs *= force_reduction_factor / distances
                
                # Find how much "force" must be applied to the coordinates by computing
                # the dot product of the force vector and the centroid deltas.
                new_coordinates += Fijs.dot(new_coordinates - centroids)

            # Set the polygon 
            geometry[row] = Polygon(coordinates, holes = item.interiors)
            
    return gpd.geoseries.GeoSeries(geometry)

In [ ]:
exploded.to_crs('+proj=cea')
pop_cart=cartogram(exploded['geometry'], exploded['pop_est'])

In [ ]:
def cartogram(arg_polygons, arg_values, itermax=5, max_size_error=1.0001, epsilon=0.01, verbose=False):
    """
    Generate an area equalizing contiguous cartogram based on the algorithm by (J. Oougenik et al., 1985).
    
    Note: The current function does not include interior boundaries when distorting the polygons!
          This is due to shapely's current way of extracting boundary coordinates which make it 
          cumbersome to separate interior points from exterior points.
    
    :param arg_polygons: geopandas.geoseries.GeoSeries Series of shapely.geometry.Polygon objects.
    :param arg_values: (geo)pandas.Series Series of floating point values.
    :param itermax: int (Optional, default=5) Maximum amount of iterations to perform adjusting coordinates.
    :param max_size_error: float (Optional, default=1.0001) A maximum accuracy until terminating the procedure.
    :param epsilon: float (Optional, default=0.01) Scalar to prevent zero division errors.
    :param verbose: bool (Optional, default=False) Whether to print out intermediary progress. 
    
    :returns: geopandas.geoseries.GeoSeries Copy of :arg_polygons: with the adjusted coordinates.
    
    :references: Dougenik, J.A., Chrisman, N.R. and Niemeyer, D.R. (1985), 
                 AN ALGORITHM TO CONSTRUCT CONTINUOUS AREA CARTOGRAMS*. 
                 The Professional Geographer, 37: 75-81. doi:10.1111/j.0033-0124.1985.00075.x 
    
    :see: Implementation of the same algorithm in R (available on CRAN): https://github.com/sjewo/cartogram
    """    
    polygons = arg_polygons.copy().values
    values = arg_values.copy().values
    
    total_value = values.sum()
    mean_size_error = 100
    
    for iteration in range(itermax):
        if mean_size_error < max_size_error:
            break
        
        # This statement unpacks the centroid Point object to np.array and
        # creates a n x 2 matrix of centroid [x, y] coordinates.
        centroids = np.array(list(map(np.array, polygons.centroid)))
        area = polygons.area
        total_area = area.sum()
        
        desired = total_area * values / total_value
        desired[desired == 0] = epsilon  # Prevent zero division.
        radius = np.sqrt(area / np.pi)
        mass = np.sqrt(desired / np.pi) - np.sqrt(area / np.pi)
        
        size_error = np.max([desired, area], axis=0) - np.min([desired, area], axis=0)
        mean_size_error = np.mean(size_error)
        force_reduction_factor = 1 / (1 + mean_size_error)
        
        if verbose:
            print("Mean size error at iteration {}: {}".format(iteration+1, mean_size_error))
        for row, polygon in enumerate(polygons):
            
            # TODO: Possibly include shapely.geometry.Polygon interior coordinates.
            
            # Some coordinates may appear twice, however, they mustn't be removed.
            # These coordinates are also adjusted, but only computed once:
            coordinates = np.matrix(polygon.exterior.coords)    # [[x1, y2], [x2, y2], ...]
            idx = np.unique(coordinates, axis=0)                # Get unique rows
            
            for k in range(len(idx)):
                # Get positions from coordinates for each unique idx.
                coord_idx = np.where((coordinates[:, 0] == idx[k,0]) & (coordinates[:,1] == idx[k, 1]))[0]
                # Only extract one using coord_idx[0] as coord_idx maps duplicate coordinates.
                new_coordinates = coordinates[coord_idx[0],:]  
                
                # Compute coordinate's euclidean distances to all centroids.
                distances = np.sqrt(np.square(centroids - new_coordinates).sum(axis=1))
                distances = np.array(distances).ravel()  # Converts matrix into flat array.
                
                # Compute force vectors
                Fijs = mass * radius / distances
                Fbij = mass * np.square(distances / radius) * (4 - 3 * distances / radius)
                Fijs[distances <= radius] = Fbij[distances <= radius]
                Fijs *= force_reduction_factor / distances
                
                # Find how much "force" must be applied to the coordinates by computing
                # the dot product of the force vector and the centroid deltas.
                new_coordinates += Fijs.dot(new_coordinates - centroids)
                
            # Set the polygon 
            polygons[row] = Polygon(coordinates, holes = polygon.interiors)
            
    return gpd.geoseries.GeoSeries(polygons)

In [ ]:
exploded = world.explode()
exploded.to_crs('+proj=cea')
exploded

In [ ]:
pop_cart=cartogram(exploded['geometry'], exploded['pop_est'],itermax=100,verbose=True)
pop_cart[0]

In [ ]:
pop_cart.plot()

In [ ]:
#Check Geometry
def compute_area(geom):
    area = geom.area
    return area

world['area'] = world['geometry'].apply(lambda x: compute_area(x))
world['pop_dens']=world['pop_est']/world['area']
world['gdp_dens']=world['gdp_md_est']/world['area']
explode_me=world.drop(['pop_est','gdp_md_est','area'],axis=1)
exploded=explode_me.explode()
exploded['area'] = exploded['geometry'].apply(lambda x: compute_area(x))
exploded['pop']=exploded['pop_dens']*exploded['area']
exploded

In [ ]:
pop_cart=cartogram(exploded['geometry'], exploded['pop'],itermax=50,verbose=True)
print(pop_cart[0])
pop_cart.plot()

In [ ]:
#exploded['geometry']=pop_cart
exploded['cart']=pop_cart
exploded.set_geometry('cart')
#print(exploded['pop'].values)
#print(pop_cart)

df = pd.DataFrame({'pop': exploded['pop'].values})
gdf = gpd.GeoDataFrame(df, geometry=pop_cart)
gdf.plot('pop', label=True)